Assamese -bytepair embedding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!pip install emot --upgrade
# to convert emojis to text
!pip install emoji
#to expand a contracted words
!pip install demoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=4a1efd89af80085039e1ec65cf01be6cdf4d4baa1f7de340cf9a38c530e033f5
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")
with open('/content/drive/MyDrive/hasoc/task4/assam/assamese.txt', encoding = 'utf-8') as f:
    assam_stopwords = f.readlines()
    for i in range(len(assam_stopwords)):
        assam_stopwords[i] = re.sub('\n','', assam_stopwords[i])
stopwords = english_stopwords + assam_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
len(stopwords)

295

In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-7-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-8-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    if len(text) == 0:
      text= '**********************'
    return text

In [ ]:
asa_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/assam/train_A_AH_HASOC2023.csv')
asa_test=pd.read_csv('/content/drive/MyDrive/hasoc/task4/assam/test_A_AH_HASOC2023.csv')

In [ ]:
asa_train['task_1'].value_counts()

HOF    2347
NOT    1689
Name: task_1, dtype: int64

In [ ]:
asa_train['clean_text'] = asa_train['text'].apply(lambda x:emo(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:remove_urls(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
asa_train[asa_train['clean_text'] =='**********************']

,S. No.,text,task_1,clean_text
2107,2108,হা হা হা হা হা হা হা হা হা,NOT,**********************


In [ ]:
asa_train['clean_text']

0                                                                                                                                      লাগিলে ইযাৰ পৰাই কলি লাগাম
1                                                                         আমি গেদা মই গেদা rolling floor laughing rolling floor laughing rolling floor laughing ।
2                                      মানুহে যেতিয়া নিজৰ জীৱন টো নিজৰ কাৰণে সমাজৰ কাৰণে দেশৰ কাৰণে পৃথিৱীৰ কাৰণে কিমান মূল্যবান বুজি নাপায় তেতিয়া এনেকুৱা হয়
3                                                                                                                   কতি খন কোন দিনা দেখা পাও তাকে বাত চায় আছো মই
4       কাপোৰ খন চাইকেল অহা মানুহজনৰ মুখত দলিয়াব লাগেনে বেছেৰাটো গুচি গল সেইকাৰণে বেয়া পায়। rolling floor laughing rolling floor laughing rolling floor laughing
                                                                                  ...                                                                            
4031                        

In [ ]:
asa_test['clean_text'] = asa_test['text'].apply(lambda x:emo(x))
asa_test['clean_text'] = asa_test['clean_text'].apply(lambda x:remove_urls(x))
asa_test['clean_text'] = asa_test['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
asa_test['clean_text']

0                                               তথাপিও ভোট বিজেপিকেই দিয়া হব সিধা কথা লাগিলে ভিক্ষাৰি হম তথাপিও ভোট বিজেপিকেই দিম
1                                                                                                মদ খাই নিজৰ কেৰিয়াৰ ধ্বংস কৰিছে।
2                                                            বিপ্লৱৰ কাইঁটীয়া পথ ইমান সহজ নহয় বাপুহঁত।সহজলভ্য জীৱন লাগে তোমালোকক
3                                                                     ক্লা ফকটীয়া ফাল্টু ঘোঁ‌ৰা চহিচ । কাম নাই ষ্টেচনত ঝাড়ু মাৰ গৈ
4       যিয়ে কোচবিহাৰ ৰাজ্যৰ বিৰোধিতা কৰে সি মোগলৰ বংশধৰ কোচবিহাৰ ৰাজ্য হোৱাৰ লগে লগে বস্তুাত ভৰাই ব্ৰহ্মপুত্ৰ নদীত পেলাই দিয়া হব।
                                                                   ...                                                            
1004                                                                                            ভাই বিয়া চিয়া পাতি আমাক পাহৰি গৈছা
1005                                                     সুদুৰ ভাই কোবাট মুৰি আচেলি

In [ ]:
asa_test.head()

,S. No.,text,clean_text
0,1,"তথাপিও ভোট বিজেপিকেই দিয়া হ'ব সিধা কথা লাগিলে ভিক্ষাৰি হ'ম তথাপিও ভোট বিজেপিকেই দিম,,",তথাপিও ভোট বিজেপিকেই দিয়া হব সিধা কথা লাগিলে ভিক্ষাৰি হম তথাপিও ভোট বিজেপিকেই দিম
1,2,মদ খাই নিজৰ কেৰিয়াৰ ধ্বংস কৰিছে।,মদ খাই নিজৰ কেৰিয়াৰ ধ্বংস কৰিছে।
2,3,বিপ্লৱৰ কাইঁটীয়া পথ ইমান সহজ নহয় ঐ বাপুহঁত।সহজলভ্য জীৱন লাগে তোমালোকক????????,বিপ্লৱৰ কাইঁটীয়া পথ ইমান সহজ নহয় বাপুহঁত।সহজলভ্য জীৱন লাগে তোমালোকক
3,4,"ক্লা , ঐ ফকটীয়া ফাল্টু ঘোঁ‌ৰা চহিচ । কাম নাই যদি ষ্টেচনত ঝাড়ু মাৰ গৈ",ক্লা ফকটীয়া ফাল্টু ঘোঁ‌ৰা চহিচ । কাম নাই ষ্টেচনত ঝাড়ু মাৰ গৈ
4,5,যিয়ে কোচবিহাৰ ৰাজ্যৰ বিৰোধিতা কৰে সি মোগলৰ বংশধৰ আৰু কোচবিহাৰ ৰাজ্য হোৱাৰ লগে লগে বস্তুাত ভৰাই ব্ৰহ্মপুত্ৰ নদীত পেলাই দিয়া হব।,যিয়ে কোচবিহাৰ ৰাজ্যৰ বিৰোধিতা কৰে সি মোগলৰ বংশধৰ কোচবিহাৰ ৰাজ্য হোৱাৰ লগে লগে বস্তুাত ভৰাই ব্ৰহ্মপুত্ৰ নদীত পেলাই দিয়া হব।


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199770 sha256=f20cb4f26943bda348b7364527b9819cffd179dce29a4f21b0617d366abb60e1
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import gzip
import numpy as np

In [ ]:
embedding_index_asa= {}
asavocab = []

with open('/content/drive/MyDrive/hasoc/task4/assam/as.wiki.bpe.vs100000.d300.w2v.txt', encoding='utf-8') as f:
    for line in f:
        values = line.rstrip().split(' ')
        word = values[0]
        asavocab.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index_asa[word] = coefs

In [ ]:
embedding_index_eng= {}
engvocab = []

with open('/content/drive/MyDrive/hasoc/task4/assam/en.wiki.bpe.vs200000.d300.w2v.txt', encoding='utf-8') as f:
    for line in f:
        values = line.rstrip().split(' ')
        word = values[0]
        engvocab.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index_eng[word] = coefs

In [ ]:
len(asavocab)

100001

In [ ]:
len(engvocab)

200001

In [ ]:
def get_vectors(sentence):
    sentvec = np.zeros(300)
    zeros = np.zeros(300)

    for word in sentence.split():
        if word in engvocab:
            temp = embedding_index_eng[word]
        elif word in asavocab:
            temp = embedding_index_asa[word]
        else:
            temp = zeros
        # print('word : ', word, ' ttt : ', temp)
        sentvec += temp

    sentvec = sentvec / len(sentence.split())
    return sentvec

In [ ]:
asa_train['vectors'] = asa_train['clean_text'].apply(lambda x:get_vectors(x))
asa_test['vectors'] = asa_test['clean_text'].apply(lambda x:get_vectors(x))

In [ ]:
type(asa_train['vectors'])

pandas.core.series.Series

In [ ]:
zeros=([0.]*300)
zeros=np.array(zeros)

In [ ]:
asa_train['clean_text'][2107]=zeros

<ipython-input-36-a95c6568a06b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asa_train['clean_text'][2107]=zeros


In [ ]:
X_train=asa_train['vectors'].to_list()
x_test=asa_test['vectors'].to_list()

In [ ]:
y_train =asa_train['task_1'].to_list()

In [ ]:
type(y_train)

list

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
svm_model = SVC(kernel='linear')

In [ ]:
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
y_pred1 = svm_model.predict(x_test)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred1, columns=['y_pred1'])
Submisssion_assam = pd.DataFrame()
Submisssion_assam['S. No.'] = asa_test['S. No.']
Submisssion_assam['task_1'] = y_pred_df
Submisssion_assam.to_csv('/content/drive/MyDrive/MUCS_test_assam_run11.csv',index = None)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)], voting='hard')
eclf = eclf.fit(X_train, y_train)
y_pred2= eclf.predict(x_test)
print(y_pred2)

['HOF' 'HOF' 'HOF' ... 'HOF' 'HOF' 'HOF']


In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred1, columns=['y_pred2'])
Submisssion_assam = pd.DataFrame()
Submisssion_assam['S. No.'] = asa_test['S. No.']
Submisssion_assam['task_1'] = y_pred_df
Submisssion_assam.to_csv('/content/drive/MyDrive/MUCS_test_assam_run12.csv',index = None)